### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [41]:
import os
from dotenv import load_dotenv
load_dotenv()   

True

In [42]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re known for their ability to mimic human speech, right? I remember seeing parrots on TV or in videos where they repeat words, sometimes even phrases. But why do they do that? Is it just to imitate, or is there a deeper reason?\n\nFirst, I should consider their natural behavior. In the wild, parrots live in social groups. Maybe talking helps them communicate with each other. I\'ve heard that some birds use calls to signal to others in their flock, like to warn of danger or locate each other. So, maybe parrots use vocalizations for similar purposes. But why would they mimic human speech specifically?\n\nI think it has something to do with their intelligence. Parrots are considered highly intelligent birds. They can solve problems and have good memory. Maybe their ability to mimic sounds is an extension of their cognitive abilities. They might learn to as

In [43]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"

In [44]:
model_with_tools = model.bind_tools([get_weather])

In [45]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. The required parameter is location, which should be a string. So I\'ll call get_weather with location set to "Boston". Make sure the JSON is correctly formatted with the name and arguments.\n', 'tool_calls': [{'id': '5hxcttr0c', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 154, 'total_tokens': 244, 'completion_time': 0.129789406, 'completion_tokens_details': {'reasoning_tokens': 66}, 'prompt_time': 0.006702315, 'prompt_tokens_details': None, 'queue_time': 0.048165475, 'total_time': 0.136491721}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run-

## Tool Execution loop

In [46]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny. ☀️ Let me know if you need more details!
